In [ ]:
import tweepy
from tweepy.auth import OAuthHandler
from tweepy import Stream
#from tweepy.streaming import StreamListener
import socket
import json

In [ ]:
# request to get credentials at http://apps.twitter.com
consumer_key    = '3XwgFgsXucaFXOrkZtlwvxV5O'
consumer_secret = 'LNnTcxibKalylVImljDsKfkRqb6WHD8I7hdSYt7Pm7VaCetqZm'
access_token    = '608048932-AM21gcwLORlm5j0514DwPL850byLb7Gs04bPZ6Mn'
access_secret   = 'QtnJ8mV75peV7L7Kd4DZKogMRIxzjk5XFOPtZiz62NoM3'

In [ ]:
# create this class that inherits from the StreamListener in tweepy StreamListener
class TweetsListener(tweepy.Stream):

    def __init__(self, csocket):
        self.client_socket = csocket
    # override the on_data() function in StreamListener
    def on_data(self, data):
        try:
            message = json.loads( data )
            print( message['text'].encode('utf-8') )
            self.client_socket.send( message['text'].encode('utf-8') )
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def if_error(self, status):
        print(status)
        return True

In [ ]:
def send_tweets(c_socket):
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    
    twitter_stream = Stream(auth, TweetsListener(c_socket))

In [ ]:
if __name__ == "__main__":
    new_skt = socket.socket()         # initiate a socket object
    #host = "127.0.0.1"     # local machine address
    #port = 5555                 # specific port for your service.
    new_skt.bind((host, port))        # Binding host and port

    print("Now listening on port: %s" % str(port))

    new_skt.listen(5)                 #  waiting for client connection.
    c, addr = new_skt.accept()        # Establish connection with client. it returns first a socket object,c, and the address bound to the socket

    print("Received request from: " + str(addr))
    # and after accepting the connection, we will send the tweets through the socket
    send_tweets(c)

In [ ]:
import findspark
findspark.find()
import pyspark

In [ ]:
# import necessary packages
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [ ]:
sc = SparkContext()
# initiate the StreamingContext with 10 second batch interval.
ssc = StreamingContext(sc, 10)
# initiate our sqlcontext
sqlContext = SQLContext(sc)

In [ ]:
# initiate streaming text from a TCP (socket) source:
socket_stream = ssc.socketTextStream(host, port)
# lines of tweets with socket_stream window of size 60, or 60 #seconds windows of time
lines = socket_stream.window(60)

In [ ]:
#### This section is optional and can be changed based on what we aime to crawl such as trending hashtags ###
# just a tuple to assign names
from collections import namedtuple
fields = ("hashtag", "count" )
Tweet = namedtuple( 'Tweet', fields )
# here we apply different operations on the tweets and save them to #a temporary sql table
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  # Checks for    hashtag calls  
  .filter( lambda word: word.lower().startswith("#") ) 
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) 
 # Stores in a Tweet Object
  .map( lambda rec: Tweet( rec[0], rec[1] ) )
 # Sorts Them in a dataframe
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )
 # Registers only top 10 hashtags to a table.
  .limit(10).registerTempTable("tweets") ) )

In [ ]:
# start streaming and wait couple of minutes to get enought tweets
ssc.start()

In [ ]:
# import libraries to visualize the results
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
get_ipython().run_line_magic('matplotlib', 'inline')
count = 0
while count < 5:
    
    time.sleep(5)
    top_10_tags = sqlContext.sql( 'Select hashtag, count from tweets' )
    top_10_df = top_10_tags.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="hashtag", data=top_10_df)
    plt.show()
    count = count + 1
    print(count)

In [ ]:
# stop streaming and wait couple of minutes to get enought tweets
ssc.stop()

In [ ]:
# Saving the tweets as a JSON file

import json
json_data = []
json_data.append(item._json)
with open('tweets.json', 'w') as json_file:
    json.dump(json_data, json_file)